# Homework 12

https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning

* Implement a mini batch functionality to train a regressor.
    - (Optional) If anyone want to do this in a pipeline can do this: https://koaning.github.io/tokenwiser/api/pipeline.html

* Save model, load the model again and test it on `X_test`

In [1]:
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as npipeline
from sklearn.linear_model import SGDRegressor
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    df = df.sample(5000, random_state=100).reset_index(drop=True)
    
    y = df['sellingprice']
    df.drop('sellingprice', axis=1, inplace=True)
    X = df
    
    return X,y

def partial_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
   
    while(True):
        yield df.sample(100).reset_index(drop=True)
        
gen = partial_df()

In [3]:
X_test, y_test = test_df()

In [4]:
X_test, y_test

(      year       make           model      trim         body transmission  \
 0     2008  Chevrolet           Tahoe        LS          SUV    automatic   
 1     2011   Infiniti         G Sedan      G37x      G Sedan    automatic   
 2     2012     Toyota            RAV4      Base          suv    automatic   
 3     2014      Dodge         Avenger        SE        Sedan    automatic   
 4     2011       Ford           Focus       SEL        Sedan    automatic   
 ...    ...        ...             ...       ...          ...          ...   
 4995  2014       Ford           F-150        XL  regular cab    automatic   
 4996  2013       Ford          Fusion  Titanium        Sedan    automatic   
 4997  2011       Jeep  Grand Cherokee    Laredo          SUV    automatic   
 4998  2014       Ford          Fusion        SE        Sedan    automatic   
 4999  2014  Chevrolet          Impala        LT        Sedan    automatic   
 
                     vin state condition  odometer   color int

In [5]:
# each time you call this you will get a new slice of the dataframe.
next(gen)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2012,Chevrolet,Equinox,LT,SUV,automatic,2gnfleek0c6305333,tx,3.4,74234.0,blue,black,ge fleet services for itself/servicer,13000,12700,Tue Dec 23 2014 14:00:00 GMT-0800 (PST)
1,2012,Ford,Focus,SE,Sedan,NaN,1fahp3f29cl385186,pa,4.3,37061.0,red,black,r hollenshead auto sales inc,9500,10400,Fri Jan 16 2015 01:00:00 GMT-0800 (PST)
2,2012,Volkswagen,Jetta,S,sedan,automatic,3vw2k7aj0cm390938,nj,2.9,62061.0,black,gray,the hertz corporation,7950,7500,Wed Jun 10 2015 02:30:00 GMT-0700 (PDT)
3,2015,Toyota,Tacoma,PreRunner V6,Double Cab,automatic,3tmju4gn4fm178212,oh,4.9,12542.0,silver,gray,no limit imports inc,26500,25000,Tue Jun 30 2015 06:30:00 GMT-0700 (PDT)
4,2014,Ford,Escape,SE,SUV,automatic,1fmcu0gx9euc09012,ma,3.4,25555.0,gray,black,lease plan usa,16550,14100,Tue Jun 16 2015 02:30:00 GMT-0700 (PDT)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2013,Toyota,Camry Hybrid,LE,Sedan,automatic,4t1bd1fk4du081851,ca,4.9,30732.0,black,gray,ge fleet services for itself/servicer,17200,19500,Tue Jan 27 2015 04:00:00 GMT-0800 (PST)
96,2014,Jeep,Compass,Sport,SUV,automatic,1c4njcba4ed882896,ga,2.1,5683.0,silver,black,"velocity auto broker, inc",15300,11900,Tue Mar 10 2015 02:30:00 GMT-0700 (PDT)
97,2004,Volvo,C70,LPT,Convertible,automatic,yv1nc63d74j062147,ca,2,101873.0,gray,gray,cal max fine cars,2175,700,Tue Jan 27 2015 04:00:00 GMT-0800 (PST)
98,2011,Nissan,Pathfinder,LE,SUV,automatic,5n1ar1nn5bc600935,tx,3.5,41121.0,white,beige,tdaf remarketing,18150,18400,Thu May 28 2015 03:30:00 GMT-0700 (PDT)


In [6]:
next(gen).columns

Index(['year', 'make', 'model', 'trim', 'body', 'transmission', 'vin', 'state',
       'condition', 'odometer', 'color', 'interior', 'seller', 'mmr',
       'sellingprice', 'saledate'],
      dtype='object')

In [7]:
# Define the missing value imputation strategies
numeric_features = ['year', 'odometer']
categorical_features = ['make', 'model', 'trim', 'body', 'transmission', 'vin', 'state', 'condition', 'color', 'interior', 'seller', 'saledate']
numeric_imputer = SimpleImputer(strategy='median')
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Define the transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', numeric_imputer),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', categorical_imputer),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Fit the preprocessor on the training data
preprocessor.fit(X_test)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['year', 'odometer']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['make', 'model', 'trim', 'body',
                                  'transmission', 'vin', 'state', 'condition',
                                  'color', 'interior', 'seller', 'saledate'])])

In [8]:
import numpy as np
from xgboost import XGBRegressor

In [11]:
"""
# Create the XGBRegressor
regressor = XGBRegressor(random_state=100)

# Mini-batch training loop
batch_size = 50
num_batches = 100
n_epoch = 100

for epoch in range(n_epoch):
    for _ in range(num_batches):
        batch_df = next(gen)
        X_batch = batch_df.drop('sellingprice', axis=1)
        y_batch = batch_df['sellingprice']

        # Apply preprocessor to the mini-batch
        X_batch_processed = preprocessor.transform(X_batch)

        # Handle missing values in the mini-batch
        X_batch_processed = np.nan_to_num(X_batch_processed)

        regressor.fit(X_batch_processed, y_batch)"""

"\n# Create the XGBRegressor\nregressor = XGBRegressor(random_state=100)\n\n# Mini-batch training loop\nbatch_size = 50\nnum_batches = 100\nn_epoch = 100\n\nfor epoch in range(n_epoch):\n    for _ in range(num_batches):\n        batch_df = next(gen)\n        X_batch = batch_df.drop('sellingprice', axis=1)\n        y_batch = batch_df['sellingprice']\n\n        # Apply preprocessor to the mini-batch\n        X_batch_processed = preprocessor.transform(X_batch)\n\n        # Handle missing values in the mini-batch\n        X_batch_processed = np.nan_to_num(X_batch_processed)\n\n        regressor.fit(X_batch_processed, y_batch)"

# I tried using various regressor models but they are taking too much computational time and power. Finally I took the SGD Regressor by referring the Week 12 online learning. And also it is best fit and taking less computational time.



In [10]:
# Create the SGDRegressor
regressor = SGDRegressor(loss='squared_error', random_state=100)

# Mini-batch training loop
batch_size = 50
num_batches = 100
n_epoch = 100

for epoch in range(n_epoch):
    for _ in range(num_batches):
        batch_df = next(gen)
        X_batch = batch_df.drop('sellingprice', axis=1)
        y_batch = batch_df['sellingprice']

        # Apply preprocessor to the mini-batch
        X_batch_processed = preprocessor.transform(X_batch)

        # Handle missing values in the mini-batch
        X_batch_processed = np.nan_to_num(X_batch_processed)

        regressor.partial_fit(X_batch_processed, y_batch)

In [12]:
# Preprocess the test data
X_test_processed = preprocessor.transform(X_test)

# Evaluate on test data
y_pred = regressor.predict(X_test_processed)


In [13]:
# Save the model
import joblib
joblib.dump(regressor, 'regressor_model.joblib')

# Load the model
loaded_regressor = joblib.load('regressor_model.joblib')

# Preprocess the test data
X_test_processed = preprocessor.transform(X_test)

# Test the model on X_test
y_pred = loaded_regressor.predict(X_test_processed)

print(y_pred)

[14348.15585961 20190.50700553 15145.35251274 ... 19974.49024589
 19084.23987838 16212.19683595]


In [14]:
loaded_regressor.score(X_test_processed,y_test) 

0.8106424329220008

# Reference 
Week 12 notes